In [4]:
from tkinter import * 

global M_value

def add(Event): 
    s1 = e1.get()
    s2 = e2.get()
    try:
        l['text'] = float(s1) + float(s2)
    except ValueError:
        l['text'] = "Ошибка ввода, введите число"
    
    
def sub(Event):
    s1 = e1.get()
    s2 = e2.get()
    try:
        l['text'] = float(s1) - float(s2)
    except ValueError:
        l['text'] = "Ошибка ввода, введите число"
    
def mul(Event):
    s1 = e1.get()
    s2 = e2.get()
    try:
        l['text'] = float(s1) * float(s2)
    except ValueError:
        l['text'] = "Ошибка ввода, введите число"
    
def div(Event):
    s1 = e1.get()
    s2 = e2.get()
    try: 
        l['text'] = float(s1) / float(s2)
    except ZeroDivisionError:
        l['text'] = "Мы тут не делим на ноль"
    except ValueError:
        l['text'] = "Ошибка ввода, введите число"
        
def get_mem(Event):
    global Mvalue
    Mvalue = l.cget("text")
    #print(Mvalue)
    
def set_mem(Event):
    global Mvalue
    e1.delete(0,"end")
    e1.insert(0,float(Mvalue))
    
def reset(Event):
    e1.delete(0,"end")
    e2.delete(0,"end")
    l['text'] = ""
        
root = Tk() 
root.geometry("600x250")

b_add = Button(text = "+")
b_add.bind('<Button-1>', add)
b_add.grid(row=0, column=0, ipadx=10, ipady=6, padx=10, pady=10)

b_sub = Button(text = "-")
b_sub.bind('<Button-1>', sub)
b_sub.grid(row=0, column=1, ipadx=10, ipady=6, padx=10, pady=10)

b_mul = Button(text = "*")
b_mul.bind('<Button-1>', mul)
b_mul.grid(row=0, column=2, ipadx=10, ipady=6, padx=10, pady=10)

b_div = Button(text = "/")
b_div.bind('<Button-1>', div)
b_div.grid(row=1, column=0, ipadx=10, ipady=6, padx=10, pady=10)

b_mem = Button(text = "M")
b_mem.bind('<Button-1>', get_mem) #ЛКМ
b_mem.bind('<Button-3>', set_mem) #ПКМ
b_mem.grid(row=1, column=1, ipadx=10, ipady=6, padx=10, pady=10)

b_res = Button(text = "Сбросить")
b_res.bind('<Button-1>', reset)
b_res.grid(row=1, column=2, ipadx=10, ipady=6, padx=10, pady=10)

e1 = Entry(width=40)
e2 = Entry(width=40)
e1.grid(row=2, column=0, ipadx=10, ipady=6, padx=10, pady=10)
e2.grid(row=3, column=0, ipadx=10, ipady=6, padx=10, pady=10)

l = Label(fg='black')
l.grid(row=2, column=1, ipadx=10, ipady=6, padx=10, pady=10)
root.mainloop() 


Нажатие левой кнопки мыши по кнопке "М" помещает результат в память, нажатие правой, выводит в первое поле ввода.

In [8]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import tkinter as tk
from tkinter import ttk

url = "https://tradingeconomics.com"
c_list = {}    
def country_list():
    global url, c_list
    response = requests.get(url+"/matrix")
    soup = bs(response.text, 'lxml')
    table = soup.find_all('tbody')[0] # Grab the first table
    #print(table)
    for stats in table.find_all('tr'):
        rows = stats.find_all('td')
        try:
            col0 = rows[0].text.strip()
            col0_href = rows[0].find('a')['href']
            c_list[col0] = col0_href
        except:
            pass
            
    return c_list    


def param_list():
    global url
    p_list = []
    response = requests.get(url+"/united-states/indicators") #just a random country
    soup = bs(response.text, 'lxml')
        
    table = soup.find_all('tbody')[0] # Grab the first table
    #print(table)
    for stats in table.find_all('tr'):
        rows = stats.find_all('td')
        try:
            col0 = rows[0].text.strip()
            p_list.append(col0)
        except:
            pass
            
    return p_list
    
def get_value(Event):  
    country = comboC.get()
    param = comboPar.get()
    response = requests.get(url+"/"+c_list[country]+"/indicators") #just a random country
    soup = bs(response.text, 'lxml')
    table = soup.find_all('tbody')[0] # Grab the first table
    is_find = False    
    for stats in table.find_all('tr'):
        rows = stats.find_all('td')
        if (param == rows[0].text.strip()):
            #print(rows[1].text.strip())
            res['text'] = rows[1].text.strip()
            is_find = True
            break
    if (not is_find):
        res['text'] = "This parameter is unavailable for " + country

    
if __name__ == "__main__":
    
    c_list_sh = country_list()
    p_list_sh = param_list()
    
    
    #html.get_value("Russia", "Currency")
    
    
    app = tk.Tk() 
    app.geometry('400x200')
    
    labelCTop = tk.Label(app,
                    text = "Choose country")
    labelCTop.grid(column=0, row=0)

    comboC = ttk.Combobox(app, 
                          values=list(c_list_sh.keys()))

    comboC.grid(column=0, row=1)
    comboC.current(0)
    

    labelTop = tk.Label(app,
                    text = "Choose parameter")
    labelTop.grid(column=1, row=0)

    comboPar = ttk.Combobox(app, 
                            values=p_list_sh)

    comboPar.grid(column=1, row=1)
    comboPar.current(0)
    
    b_get = Button(text = "Get value")
    b_get.bind('<Button-1>', get_value)
    b_get.grid(column = 0, row = 3)
    
    res = Label(fg='black')
    res.grid(column = 1, row = 3, ipadx=10, ipady=6, padx=10, pady=10)
    


    app.mainloop()
    
    